<a href="https://colab.research.google.com/github/pushyag1/BigData_PySpark/blob/main/Pyspark_All_codes_Compiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title PySpark Setup(run me!)
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark



Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bio

In [ ]:
#@title Dynamic Input(select your choices and run the cell)
Date_to_calculate_age = '2021-02-01' #@param {type:"date"}
Choose_a_model_version = 'V24' #@param ["V22","V24", "RX", "ESRD-P1","ESRD-P2"]
Choose_a_model_year = '2021' #@param ["2020","2021"]
Sex_Age_edits_required = 'Yes' #@param ["Yes","No"]

In [ ]:
#@title Age Calculation
import pandas 
import os
import pyspark.sql.functions as func
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import struct
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf


#Loading Persons file
df = pandas.read_excel ("person file - all inputs.xlsx")
df1=df.drop(['Unnamed: 8', 'Unnamed: 9','Unnamed: 10'], axis=1)
df1 = spark.createDataFrame(df1)

df2=df1.withColumn("Age",(datediff(to_date(lit(Date_to_calculate_age)),to_date("DOB","yyyy/MM/dd")))/366)
df3 = df2.withColumn("Age", func.round(df2["Age"], 2).cast('integer'))

#Condition for AGE and OREC
def func(Age, OREC):
  if Age == 64 and OREC == 0:
    return 65
  elif Age < 0 :
    return 0
  return Age


func_udf = udf(func, IntegerType())
df4 = df3.withColumn('new_column',func_udf(df3['Age'], df3['OREC']))

drop_list = [ 'Age',]
sdf5=df4.select([column for column in df4.columns if column not in drop_list])

df_final = sdf5.withColumnRenamed("new_column", "Age")

df_final.show()


+---------+-------------------+------+----+--------+-------+----+-----+---+
|Person ID|                DOB|GENDER|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|
+---------+-------------------+------+----+--------+-------+----+-----+---+
| 95307787|1944-03-23 00:00:00|     1|   0|       1|      0|   1|    1| 76|
| 67928991|1930-05-29 00:00:00|     2|   0|       1|      0|   0|    1| 90|
|124623786|1948-07-30 00:00:00|     1|   0|       1|      0|   0|    0| 72|
| 73717666|1935-06-12 00:00:00|     2|   0|       1|      0|   0|    1| 85|
|111693065|1927-05-03 00:00:00|     2|   0|       1|      0|   0|    0| 93|
|120898337|1930-11-02 00:00:00|     2|   0|       1|      0|   1|    0| 90|
|124116805|1933-02-14 00:00:00|     2|   0|       1|      0|   1|    0| 87|
| 14464261|1962-07-27 00:00:00|     2|   1|       1|      0|   1|    1| 58|
| 59011425|1929-06-10 00:00:00|     1|   0|       1|      0|   1|    1| 91|
|155756563|1927-07-25 00:00:00|     2|   0|       1|      0|   0|    1| 93|
| 24692817|1

###Please Refer above df_final(dataframe for further calculation)-Himanshu

In [ ]:
#@title Demography Variable

import pandas as pd
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

df_age = df_final.toPandas()
df_final_copy = df_age.copy()
data = spark.createDataFrame(df_final_copy)

demo = (spark.read.format("csv").options(header="true").load("/content/Demography Variable Calculations.csv"))

demo = demo.filter((demo['Version'] == Choose_a_model_version) & (demo['Year'] == Choose_a_model_year))
data = data.withColumn("Model Year", lit(str(Choose_a_model_year)))
data = data.withColumn("Model Version", lit(str(Choose_a_model_version)))

for i in demo.collect():
  data = data.withColumn(i[2], expr(i[3]))

data.show()

#data.toPandas().to_excel('v24'+ '.xlsx')

+---------+-------------------+------+----+--------+-------+----+-----+---+----------+-------------+------+-----+------+------+------+------+------+------+------+------+------+------+------+----------+-----+------+------+------+------+------+------+------+------+------+------+------+----------------+----------------+---------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+--------+--------+--------+--------+--------+--------+----------------+----------------+------+
|Person ID|                DOB|GENDER|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|Model Year|Model Version|DISABL|F0_34|F35_44|F45_54|F55_59|F60_64|F65_69|F70_74|F75_79|F80_84|F85_89|F90_94|F95_GT|LTIMCAID_O|M0_34|M35_44|M45_54|M55_59|M60_64|M65_69|M70_74|M75_79|M80_84|M85_89|M90_94|M95_GT|MCAID_NORIGDIS  |MCAID_ORIGDIS   |NE_ORIGDS|NEF0_34|NEF35_44|NEF45_54|NEF55_59|NEF60_64|NE

In [ ]:
#@title CC_Mapping

import pandas as pd
import numpy as np
import time
from pyspark.sql.functions import length,col,trim
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col

df_diagnosis = pd.read_csv('Diagnosis Input File.csv')
df_diagnosis = spark.createDataFrame(df_diagnosis)
df_diagnosis = df_diagnosis.withColumn('DIAGNOSIS CODE',trim(col("DIAGNOSIS CODE")))

df_CC_mapping = pd.read_csv('CC Mapping.csv')
label_schema = StructType([
    StructField("YEAR", StringType()),
    StructField("VERSION", StringType()),
    StructField("DIAGNOSIS CODE", StringType()),
    StructField("CC", IntegerType())
])
df_CC_mapping = spark.createDataFrame(df_CC_mapping,schema= label_schema)


start=time.time()
# for year in model_years:
#   for version in versions:
cond1= col('YEAR')== str(Choose_a_model_year)
cond2= col('VERSION')== str(Choose_a_model_version)
df_temp= df_CC_mapping.where(cond1 & cond2)

# df_temp.show(5)
df_temp= df_temp.toDF('YEAR','VERSION','D_DIAG CODE','CC')

h= df_temp.join(df_diagnosis,(df_diagnosis["DIAGNOSIS CODE"] == df_temp["D_DIAG CODE"]),how='right')  # Mtachin diagnosis codes from diagnosis input file and CC_mapping file

unique_HCC= sorted([i.CC for i in df_temp.select('CC').distinct().collect()]) #Finding all unique values of HCC and storing it in a list to generate HCC columns later

CC_list = h.select("CC").rdd.flatMap(lambda x: x).collect()

p_id = h.select("PERSON ID").rdd.flatMap(lambda x: x).collect()

diag_code= h.select("DIAGNOSIS CODE").rdd.flatMap(lambda x: x).collect()


df_main= pd.DataFrame()
df_main['PERSON ID']= p_id
df_main['DIAGNOSIS CODE']= diag_code 

df_main['HCC']= CC_list  # CC values obtained after mapping from Part 2.
df_main['HCC'] = list(df_main['HCC'].astype(np.float).astype("Int32"))

df_main['MODEL YEAR']= str(Choose_a_model_year)
df_main['Version']=str(Choose_a_model_version)

# df_main
if 'RX' in Choose_a_model_version:
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('RXHCC'+str(value)) # Example: RXHCC1,RXHCC2,RXHCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'RXHCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')


else:
  # Part 5: Creating new columns with column names as HCC values obtained from Part 4
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('HCC'+str(value)) # Example: HCC1,HCC2,HCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'HCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')

print(time.time()-start)
df_main.head()

39.94522666931152


,PERSON ID,DIAGNOSIS CODE,HCC,MODEL YEAR,Version,HCC1,HCC2,HCC6,HCC8,HCC9,HCC10,HCC11,HCC12,HCC17,HCC18,HCC19,HCC21,HCC22,HCC23,HCC27,HCC28,HCC29,HCC33,HCC34,HCC35,HCC39,HCC40,HCC46,HCC47,HCC48,HCC51,HCC52,HCC54,HCC55,HCC56,HCC57,HCC58,HCC59,HCC60,HCC70,...,HCC83,HCC84,HCC85,HCC86,HCC87,HCC88,HCC96,HCC99,HCC100,HCC103,HCC104,HCC106,HCC107,HCC108,HCC110,HCC111,HCC112,HCC114,HCC115,HCC122,HCC124,HCC134,HCC135,HCC136,HCC137,HCC138,HCC157,HCC158,HCC159,HCC161,HCC162,HCC166,HCC167,HCC169,HCC170,HCC173,HCC176,HCC186,HCC188,HCC189
0,59011425,N179,135,2021,V24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,71251632,N179,135,2021,V24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,157598647,N179,135,2021,V24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049180,N179,135,2021,V24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,123742578,N179,135,2021,V24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## @**Pushyami**- Based on the model year and version given in the input, the final dataframe obtained is df_main. You can use this dataframe to go ahead and make the CC overrides.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Index/data.xlsx')
df

In [ ]:
df.rename(columns={'DIAGNOSIS CODE':'DIAG', 'Gender':'SEX', 'Age':'AGEF'}, inplace = True)

In [ ]:
versions = dataCC['VERSION'].unique().tolist()
versions

In [ ]:
years = dataCC['MODEL YEAR'].unique().tolist()
years

In [ ]:
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [ ]:

data = (spark.read.format("csv").options(header="true").load("/content/drive/MyDrive/Index/data.csv"))


In [ ]:
df1 = df.copy()
data = spark.createDataFrame(df1)
    
overide_df = (spark.read.format("csv").options(header="true").load("/content/drive/MyDrive/Index/CC Override Rules.csv"))
overide_df=overide_df.withColumnRenamed("MODEL YEAR","MODEL_YEAR")
overide_df = overide_df.filter((overide_df.VERSION.isin(versions))&(overide_df.MODEL_YEAR.isin(years)))
for i in overide_df.collect():
  data = data.withColumn('HCC', expr(i[3]))
data.toPandas().to_excel('Merged_CCOveride.xlsx')  
    

In [ ]:
df_CCmerged = pd.read_excel('/content/drive/MyDrive/Index/Merged_CCOveride.xlsx')
df_CCmerged

In [ ]:
df=df.drop(columns=['Unnamed: 0',	'Unnamed: 0.1',	'Unnamed: 0_x'])

In [ ]:
df_2021=df[df['MODEL YEAR']== 2021]
df_2021 = df_2021.reset_index(drop=True)
df_2021

In [ ]:
df_2020=df[df['MODEL YEAR']!= 2021]
df_2020 = df_2020.reset_index(drop=True)
df_2020

In [ ]:
dfList=[df_2020,df_2021]

In [ ]:
result = []
for dat in dfList:
  dat = result.append(dat.groupby(['PERSON ID','AGEF','SEX','MODEL YEAR'] ).agg(
     HCC1 = ('HCC1','max'),HCC2 = ('HCC2','max'),HCC6 = ('HCC6','max'),HCC9 = ('HCC9','max'),HCC10 = ('HCC10','max'),HCC11 = ('HCC11','max'),HCC12 = ('HCC12','max'),
     HCC17 = ('HCC17','max'),HCC18 = ('HCC18','max'),HCC19 = ('HCC19','max'),HCC21 = ('HCC21','max'),HCC22 = ('HCC22','max'),HCC23 = ('HCC23','max'),HCC27 = ('HCC27','max'),
     HCC28 = ('HCC28','max'),HCC29 = ('HCC29','max'),HCC33 = ('HCC33','max'),HCC34 = ('HCC34','max'),HCC35 = ('HCC35','max'),HCC39 = ('HCC39','max'),HCC40 = ('HCC40','max'),
     HCC46 = ('HCC46','max'),HCC47 = ('HCC47','max'),HCC48 = ('HCC48','max'),HCC51 = ('HCC51','max'),HCC52 = ('HCC52','max'),HCC54 = ('HCC54','max'),HCC55 = ('HCC55','max'),
     HCC57 = ('HCC57','max'),HCC58 = ('HCC58','max'),HCC70 = ('HCC70','max'),HCC71 = ('HCC71','max'),HCC72 = ('HCC72','max'),HCC73 = ('HCC73','max'),HCC74 = ('HCC74','max'),																	
     HCC75 = ('HCC75','max'),HCC76 = ('HCC76','max'),HCC77 = ('HCC77','max'),HCC78 = ('HCC78','max'),HCC79 = ('HCC79','max'),HCC80 = ('HCC80','max'),HCC82 = ('HCC82','max'),
     HCC83 = ('HCC83','max'),HCC84 = ('HCC84','max'),HCC85 = ('HCC85','max'),HCC86 = ('HCC86','max'),HCC87 = ('HCC87','max'),HCC88 = ('HCC88','max'),HCC96 = ('HCC96','max'),
     HCC99 = ('HCC99','max'), HCC100 = ('HCC100','max'),HCC103 = ('HCC103','max'),HCC104 = ('HCC104','max'),HCC106 = ('HCC106','max'),HCC107 = ('HCC107','max'),HCC108 = ('HCC108','max'),
     HCC110 = ('HCC110','max'),HCC111 = ('HCC111','max'),HCC112 = ('HCC112','max'),HCC114 = ('HCC114','max'),HCC115 = ('HCC115','max'),HCC122 = ('HCC122','max'),HCC124 = ('HCC124','max'),
     HCC134 = ('HCC134','max'),HCC135 = ('HCC135','max'),HCC136 = ('HCC136','max'),HCC137 = ('HCC137','max'),HCC138 = ('HCC138','max'),HCC139 = ('HCC139','max'),HCC140 = ('HCC140','max'),HCC141 = ('HCC141','max'),
     HCC157 = ('HCC157','max'),HCC158 = ('HCC158','max'),HCC159 = ('HCC159','max'),HCC160 = ('HCC160','max'),HCC161 = ('HCC161','max'),HCC162 = ('HCC162','max'),HCC166 = ('HCC166','max'),
     HCC167 = ('HCC167','max'),HCC169 = ('HCC169','max'),HCC170 = ('HCC170','max'),HCC173 = ('HCC173','max'),HCC176 = ('HCC176','max'),HCC186 = ('HCC186','max'),HCC188 = ('HCC188','max'),
     HCC189 = ('HCC189','max'),OREC = ('OREC','max'),NEMCAID = ('NEMCAID','max'),ESRD = ('ESRD','max'),MCAID = ('MCAID','max'),GE65_DUR4_9 = ('GE65_DUR4_9','max'),LT65_DUR10PL = ('LT65_DUR10PL','max'),LT65_DUR4_9 = ('LT65_DUR4_9','max'),																	
     NEF85_GT = ('NEF85_GT','max'),NEM85_GT = ('NEM85_GT','max'),ORIGDS = ('ORIGDS','max'), Originally_ESRD_Female = ('Originally_ESRD_Female','max'),Originally_ESRD_Male = ('Originally_ESRD_Male','max'),OriginallyDisabled_Female = ('OriginallyDisabled_Female','max'),OriginallyDisabled_Male = ('OriginallyDisabled_Male','max'),
     MCAID_Female_Aged = ('MCAID_Female_Aged','max'),MCAID_Male_Aged = ('MCAID_Male_Aged','max'),MCAID_Female_NonAged = ('MCAID_Female_NonAged','max'),MCAID_Male_NonAged = ('MCAID_Male_NonAged','max'),MCAID_NORIGDIS = ('MCAID_NORIGDIS','max'),MCAID_NORIGDIS_G = ('MCAID_NORIGDIS_G','max'),MCAID_O = ('MCAID_O','max'),
     MCAID_ORIGDIS = ('MCAID_ORIGDIS','max'), MCAID_ORIGDIS_G = ('MCAID_ORIGDIS_G','max'),NE_ORIGDS = ('NE_ORIGDS','max'),NE_ORIGDS_G = ('NE_ORIGDS_G','max'),KIDNEY_ONLY_1M = ('KIDNEY_ONLY_1M','max'),KIDNEY_ONLY_2M = ('KIDNEY_ONLY_2M','max'),KIDNEY_ONLY_3M = ('KIDNEY_ONLY_3M','max'),
     DISABL = ('DISABL','max'),F0_34 = ('F0_34','max'),NEF0_34 = ('NEF0_34','max'),F35_44 = ('F35_44','max'),NEF35_44 = ('NEF35_44','max'),F45_54 = ('F45_54','max'),NEF45_54 = ('NEF45_54','max'),
     F55_59 = ('F55_59','max'),NEF55_59 = ('NEF55_59','max'),F60_64 = ('F60_64','max'),F65_69 = ('F65_69','max'),F70_74 = ('F70_74','max'),NEF70_74 = ('NEF70_74','max'),F75_79 = ('F75_79','max'),NEF75_79 = ('NEF75_79','max'),
     F80_84 = ('F80_84','max'),NEF80_84 = ('NEF80_84','max'),F85_89 = ('F85_89','max'),NEF85_89 = ('NEF85_89','max'),F90_94 = ('F90_94','max'),NEF90_94 = ('NEF90_94','max'),F95_GT = ('F95_GT','max'),
     NEF95_GT = ('NEF95_GT','max'),M0_34 = ('M0_34','max'),NEM0_34 = ('NEM0_34','max'),M35_44 = ('M35_44','max'),NEM35_44 = ('NEM35_44','max'),M45_54 = ('M45_54','max'),NEM45_54 = ('NEM45_54','max'),
     M55_59 = ('M55_59','max'),NEM55_59 = ('NEM55_59','max'),M60_64 = ('M60_64','max'),M65_69 = ('M65_69','max'),M70_74 = ('M70_74','max'),NEM70_74 = ('NEM70_74','max'),M75_79 = ('M75_79','max'),
     NEM75_79 = ('NEM75_79','max'),M80_84 = ('M80_84','max'),NEM80_84 = ('NEM80_84','max'),M85_89 = ('M85_89','max'),NEM85_89 = ('NEM85_89','max'),M90_94 = ('M90_94','max'),NEM90_94 = ('NEM90_94','max'),M95_GT = ('M95_GT','max'),NEM95_GT = ('NEM95_GT','max'),
     NEF60_64 = ('NEF60_64','max'),NEF65 = ('NEF65','max'),NEF66 = ('NEF66','max'),NEF67 = ('NEF67','max'),NEF68 = ('NEF68','max'),NEF69 = ('NEF69','max'),NEM60_64 = ('NEM60_64','max'),
     NEM65 = ('NEM65','max'),NEM66 = ('NEM66','max'),NEM67 = ('NEM67','max'),NEM68 = ('NEM68','max'),NEM69 = ('NEM69','max'),NMCAID_NORIGDIS	 = ('NMCAID_NORIGDIS','max'),NMCAID_NORIGDIS_G = ('NMCAID_NORIGDIS_G','max'),NMCAID_ORIGDIS = ('NMCAID_ORIGDIS','max'),NMCAID_ORIGDIS_G = ('NMCAID_ORIGDIS_G','max'),GE65_DUR10PL_LTI = ('GE65_DUR10PL_LTI','max'),GE65_DUR10PL_NE = ('GE65_DUR10PL_NE','max'),GE65_DUR4_9_LTI = ('GE65_DUR4_9_LTI','max'),
     GE65_DUR4_9_NE = ('GE65_DUR4_9_NE','max'),LT65_DUR10PL_LTI = ('LT65_DUR10PL_LTI','max'),LT65_DUR10PL_NE = ('LT65_DUR10PL_NE','max'),LT65_DUR4_9_LTI = ('LT65_DUR4_9_LTI','max'),LT65_DUR4_9_NE = ('LT65_DUR4_9_NE','max'),RXHCC1 = ('RXHCC1','max'),RXHCC5 = ('RXHCC5','max'),
     RXHCC15 = ('RXHCC15','max'),RXHCC16 = ('RXHCC16','max'),RXHCC17 = ('RXHCC17','max'),RXHCC18 = ('RXHCC18','max'),RXHCC19 = ('RXHCC19','max'),RXHCC30 = ('RXHCC30','max'),RXHCC31 = ('RXHCC31','max'),
     RXHCC40 = ('RXHCC40','max'),RXHCC41 = ('RXHCC41','max'),RXHCC42 = ('RXHCC42','max'),RXHCC43 = ('RXHCC43','max'),RXHCC45 = ('RXHCC45','max'),RXHCC54 = ('RXHCC54','max'),RXHCC55 = ('RXHCC55','max'),
     RXHCC65 = ('RXHCC65','max'),RXHCC66 = ('RXHCC66','max'),RXHCC67 = ('RXHCC67','max'),RXHCC68 = ('RXHCC68','max'),RXHCC80 = ('RXHCC80','max'),RXHCC82 = ('RXHCC82','max'),RXHCC83 = ('RXHCC83','max'),																	
     RXHCC84 = ('RXHCC84','max'),RXHCC87 = ('RXHCC87','max'),RXHCC95 = ('RXHCC95','max'),RXHCC96 = ('RXHCC96','max'),RXHCC97 = ('RXHCC97','max'),RXHCC98 = ('RXHCC98','max'),RXHCC111 = ('RXHCC111','max'),RXHCC112 = ('RXHCC112','max'),RXHCC130 = ('RXHCC130','max'),
     RXHCC131 = ('RXHCC131','max'),RXHCC132 = ('RXHCC132','max'),RXHCC133 = ('RXHCC133','max'),RXHCC134 = ('RXHCC134','max'),RXHCC135 = ('RXHCC135','max'),RXHCC145 = ('RXHCC145','max'),RXHCC146 = ('RXHCC146','max'),
     RXHCC147 = ('RXHCC147','max'),RXHCC148 = ('RXHCC148','max'),RXHCC156 = ('RXHCC156','max'),RXHCC157 = ('RXHCC157','max'),RXHCC159 = ('RXHCC159','max'),RXHCC160 = ('RXHCC160','max'),RXHCC161 = ('RXHCC161','max'),
     RXHCC163 = ('RXHCC163','max'),RXHCC164 = ('RXHCC164','max'),RXHCC165 = ('RXHCC165','max'),RXHCC166 = ('RXHCC166','max'),RXHCC168 = ('RXHCC168','max'),RXHCC185 = ('RXHCC185','max'),RXHCC186 = ('RXHCC186','max'),
     RXHCC187 = ('RXHCC187','max'),RXHCC188 = ('RXHCC188','max'),RXHCC193 = ('RXHCC193','max'),RXHCC206 = ('RXHCC206','max'),RXHCC207 = ('RXHCC207','max'),RXHCC215 = ('RXHCC215','max'),RXHCC216 = ('RXHCC216','max'),																	
     RXHCC225 = ('RXHCC225','max'),RXHCC226 = ('RXHCC226','max'),RXHCC227 = ('RXHCC227','max'),RXHCC241 = ('RXHCC241','max'),RXHCC243 = ('RXHCC243','max'),RXHCC260 = ('RXHCC260','max'),RXHCC261 = ('RXHCC261','max'),RXHCC262 = ('RXHCC262','max'),
     RXHCC263 = ('RXHCC263','max'),RXHCC311 = ('RXHCC311','max'),RXHCC314 = ('RXHCC314','max'),RXHCC316 = ('RXHCC316','max'),RXHCC355 = ('RXHCC355','max'),RXHCC395 = ('RXHCC395','max'),RXHCC396 = ('RXHCC396','max'),RXHCC397 = ('RXHCC397','max'),
     ESRD_NORIGDIS = ('ESRD_NORIGDIS','max'),ESRD_ORIGDIS = ('ESRD_ORIGDIS','max'),F65OD = ('F65OD','max'),M65OD = ('M65OD','max'),NESRD_NORIGDIS = ('NESRD_NORIGDIS','max'),NESRD_ORIGDIS = ('NESRD_ORIGDIS','max'),NonAged = ('NonAged','max'),HCC56 = ('HCC56','max'),HCC59 = ('HCC59','max'),HCC60 = ('HCC60','max'), LTIMCAID_O = ('LTIMCAID_O','max')).reset_index())
     

print(result)		

In [ ]:
[df_2020,df_2021]=result

In [ ]:
df_2020.to_excel("data_2020.xlsx")
df_2021.to_excel("data_2021.xlsx")

-- END OF CC-OVERRIDE